In [1]:
from ultralytics import YOLO
import torch
from sklearn.cluster import KMeans
import numpy as np
from tqdm.notebook import tqdm

In [3]:
weights_path = '/usr/src/ultralytics/runs/classify/train/weights/best.pt'

model = YOLO(weights_path)

In [4]:
test_layer = model.model._modules['model'][0].conv._parameters['weight']

In [5]:
def weight_share(weights, bits=5):
    shape = weights.shape
    weights = weights.reshape(-1, 1)
    _max = weights.max()
    _min = weights.min()
    space = np.linspace(_min, _max, num=2**bits)
    kmeans = KMeans(n_clusters=len(space), random_state=0).fit(weights)
    return torch.tensor(kmeans.cluster_centers_[kmeans.labels_].reshape(shape))

In [6]:
for name, m in tqdm(model.model.named_modules()):
    if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
        clustered_weights = weight_share(m.weight)
        m.weight = torch.nn.Parameter(clustered_weights.float())

0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

In [7]:
model.val(data='imagewoof')

Ultralytics YOLOv8.0.176 🚀 Python-3.10.11 torch-2.0.1 CUDA:0 (NVIDIA A100 80GB PCIe, 81093MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 12800 gradients, 98.7 GFLOPs
train: /usr/src/datasets/imagewoof/train... found 9025 images in 10 classes ✅ 
val: /usr/src/datasets/imagewoof/val... found 3929 images in 10 classes ✅ 
test: None...
val: Scanning /usr/src/datasets/imagewoof/val... 3929 images, 0 corrupt: 100%|██████████| 3929/3929 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 246/246 [00:03<00:00, 79.26it/s] 
                   all      0.922      0.995
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /usr/src/ultralytics/runs/classify/val7


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fef2d9f0400>
fitness: 0.958513617515564
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9221175909042358, 'metrics/accuracy_top5': 0.9949096441268921, 'fitness': 0.958513617515564}
save_dir: PosixPath('/usr/src/ultralytics/runs/classify/val7')
speed: {'preprocess': 0.07888359949466806, 'inference': 0.4209097058574611, 'loss': 0.0007329141253216269, 'postprocess': 0.0006996605286436792}
top1: 0.9221175909042358
top5: 0.9949096441268921

In [32]:
clustered_n_params = 0

for param in model.model.parameters():
    clustered_n_params += param.unique().size()[0]

In [33]:
clustered_n_params

446644

In [34]:
model_src = YOLO(weights_path)

In [35]:
clustered_n_params = 0

for param in model_src.model.parameters():
    clustered_n_params += param.unique().size()[0]

In [36]:
clustered_n_params

956183